#Connect to drive

In [1]:
!git clone https://github.com/MaurizioFD/RecSys_Course_AT_PoliMi
%cd RecSys_Course_AT_PoliMi
!pip install -r requirements.txt
!python run_compile_all_cython.py

Cloning into 'RecSys_Course_AT_PoliMi'...
remote: Enumerating objects: 1493, done.
remote: Counting objects: 100% (238/238), done.
remote: Compressing objects: 100% (139/139), done.
remote: Total 1493 (delta 110), reused 225 (delta 97), pack-reused 1255
Receiving objects: 100% (1493/1493), 50.31 MiB | 25.30 MiB/s, done.
Resolving deltas: 100% (841/841), done.
/content/RecSys_Course_AT_PoliMi
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 978.0/978.0 kB 3.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 371.4/371.4 kB 29.9 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  error: subprocess-exited-with-error
  
  × pip subprocess to install backend dependencies did not run successfully.
  │ exit code: 1
  ╰─> See above for output.
  
  note: This error originates from a subprocess, and is likely not a problem with pip.
  Installing backend dependencies ... error
error: subprocess-exited-with-error

× pip subprocess t

In [2]:
from google.colab import drive
drive.mount('/gdrive')

Mounted at /gdrive


#Import Library

In [89]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
import scipy.sparse as sp
from typing import Tuple, Callable, Dict, Optional, List
from Recommenders.KNN.ItemKNNCFRecommender import ItemKNNCFRecommender
from Recommenders.Similarity.Compute_Similarity_Python import Compute_Similarity_Python
from scipy.stats import loguniform
!pip install optuna
import optuna

#Load Data

In [9]:
%cd /gdrive/MyDrive/RECSYS
data_train = pd.read_csv('data_train.csv')
data_target_users_test = pd.read_csv('data_target_users_test.csv')

/gdrive/MyDrive/RECSYS


In [10]:
data_train.columns = ["user_id", "item_id", "Interaction"]

In [11]:
print ("The number of interactions is {}".format(len(data_train)))

The number of interactions is 478730


In [12]:
userID_unique = data_train["user_id"].unique()
itemID_unique = data_train["item_id"].unique()

In [13]:
n_users = len(userID_unique)
n_items = len(itemID_unique)
n_interactions = len(data_train)

print ("Number of items\t {}, Number of users\t {}".format(n_items, n_users))
print ("Max ID items\t {}, Max Id users\t {}\n".format(max(itemID_unique), max(userID_unique)))

Number of items	 22222, Number of users	 12638
Max ID items	 22347, Max Id users	 13024



#Splitting Data

In [14]:
def preprocess_data(interactions: pd.DataFrame):
    unique_users = interactions.user_id.unique()
    unique_items = interactions.item_id.unique()

    num_users, min_user_id, max_user_id = unique_users.size, unique_users.min(), unique_users.max()
    num_items, min_item_id, max_item_id = unique_items.size, unique_items.min(), unique_items.max()

    print(num_users, min_user_id, max_user_id)
    print(num_items, min_item_id, max_item_id)

    mapping_user_id = pd.DataFrame({"mapped_user_id": np.arange(num_users), "user_id": unique_users})
    mapping_item_id = pd.DataFrame({"mapped_item_id": np.arange(num_items), "item_id": unique_items})

    interactions = pd.merge(left=interactions,
                       right=mapping_user_id,
                       how="inner",
                       on="user_id")

    interactions = pd.merge(left=interactions,
                       right=mapping_item_id,
                       how="inner",
                       on="item_id")

    return interactions


In [15]:
interactions = preprocess_data(data_train)

12638 1 13024
22222 1 22347


In [16]:
interactions

,user_id,item_id,Interaction,mapped_user_id,mapped_item_id
0,1,7,1.0,0,0
1,2,7,1.0,1,0
2,26,7,1.0,24,0
3,36,7,1.0,34,0
4,41,7,1.0,39,0
...,...,...,...,...,...
478725,12962,20368,1.0,12579,22217
478726,12985,21058,1.0,12601,22218
478727,12989,22317,1.0,12605,22219
478728,13009,22339,1.0,12624,22220


In [17]:
def dataset_splits(interactions, num_users, num_items, validation_percentage: float, testing_percentage: float):
    seed = 1234

    (user_ids_training, user_ids_test,
     item_ids_training, item_ids_test,
     interactions_training, interactions_test) = train_test_split(interactions.mapped_user_id,
                                                        interactions.mapped_item_id,
                                                        interactions.Interaction,
                                                        test_size=testing_percentage,
                                                        shuffle=True,
                                                        random_state=seed)

    (user_ids_training, user_ids_validation,
     item_ids_training, item_ids_validation,
     interactions_training, interactions_validation) = train_test_split(user_ids_training,
                                                              item_ids_training,
                                                              interactions_training,
                                                              test_size=validation_percentage,
                                                             )

    urm_train = sp.csr_matrix((interactions_training, (user_ids_training, item_ids_training)),
                              shape=(num_users, num_items))

    urm_validation = sp.csr_matrix((interactions_validation, (user_ids_validation, item_ids_validation)),
                              shape=(num_users, num_items))

    urm_test = sp.csr_matrix((interactions_test, (user_ids_test, item_ids_test)),
                              shape=(num_users, num_items))



    return urm_train, urm_validation, urm_test



In [30]:
urm_train, urm_validation, urm_test = dataset_splits(interactions,
                                                     num_users=12638,
                                                     num_items=22222,
                                                     validation_percentage=0.15,
                                                     testing_percentage=0.15)

In [19]:
urm_train

<12638x22222 sparse matrix of type '<class 'numpy.float64'>'
	with 344685 stored elements in Compressed Sparse Row format>

In [20]:
urm_validation

<12638x22222 sparse matrix of type '<class 'numpy.float64'>'
	with 38299 stored elements in Compressed Sparse Row format>

In [21]:
urm_test

<12638x22222 sparse matrix of type '<class 'numpy.float64'>'
	with 95746 stored elements in Compressed Sparse Row format>

 # Collaborative Filtering


##Item Item Similarity

In [ ]:
def vector_similarity(urm: sp.csc_matrix, shrink: int):
    item_weights = np.sqrt(
        np.sum(urm.power(2), axis=0)
    ).A.flatten()

    num_items = urm.shape[1]
    urm_t = urm.T
    weights = np.empty(shape=(num_items, num_items))
    for item_id in range(num_items):
        numerator = urm_t.dot(urm[:, item_id]).A.flatten()
        denominator = item_weights[item_id] * item_weights + shrink + 1e-6

        weights[item_id] = numerator / denominator
    np.fill_diagonal(weights, 0.0)
    return weights

In [ ]:
urm_csc = urm_train.tocsc()
shrink = 5
slice_size = 100

In [ ]:
vector_weights = vector_similarity(urm_csc[:slice_size,:slice_size], shrink)
vector_weights

#

#Build the Recommendation System

In [66]:
class CFItemKNN(object):
    def __init__(self, urm: int):
        self.urm = urm
        self.weights = None


    def fit(self, urm_train: sp.csc_matrix, topK, shrink, normalize, similarity):
        if not sp.isspmatrix_csc(urm_train):
            raise TypeError(f"We expected a CSC matrix, we got {type(urm_train)}")

        similarity_object = Compute_Similarity_Python(self.urm, shrink=shrink,
                                                  topK=topK, normalize=normalize,
                                                  similarity = similarity)

        self.weights = similarity_object.compute_similarity()

    def recommend(self, user_id: int, urm_train: sp.csr_matrix, at: Optional[int] = None, remove_seen: bool = True):
        user_profile = urm_train[user_id]

        ranking = user_profile.dot(self.weights).flatten()

        if remove_seen:
            user_profile_start = urm_train.indptr[user_id]
            user_profile_end = urm_train.indptr[user_id+1]

            seen_items = urm_train.indices[user_profile_start:user_profile_end]

            ranking[seen_items] = -np.inf

        ranking = np.flip(np.argsort(ranking))
        return ranking[:at]

In [ ]:
itemknn_recommender = CFItemKNN(shrink=50)
itemknn_recommender

In [ ]:
itemknn_recommender.fit(urm_train.tocsc(), vector_similarity)

In [ ]:
for user_id in range(10):
    print(itemknn_recommender.recommend(user_id=user_id, urm_train=urm_train, at=10, remove_seen=True))

## Top popular for the users without info

In [47]:
class TopPopRecommender(object):

    def fit(self, URM_train):

        item_popularity = np.ediff1d(URM_train.tocsc().indptr)

        # We are not interested in sorting the popularity value,
        # but to order the items according to it
        self.popular_items = np.argsort(item_popularity)
        self.popular_items = np.flip(self.popular_items, axis = 0)


    def recommend(self, user_id, at=5):

        recommended_items = self.popular_items[0:at]

        return recommended_items

#Hyperparameter Tuning


*   Number of neighbors

*   Shrinking

*   Similarity Type



EvaluatorHoldout: Ignoring 2697 (21.3%) Users that have less than 1 test interactions
EvaluatorHoldout: Ignoring 2152 (17.0%) Users that have less than 1 test interactions


In [70]:
from Recommenders.KNN.ItemKNNCFRecommender import ItemKNNCFRecommender
def objective_function(optuna_trial):

    recommender_instance = ItemKNNCFRecommender(urm_train)
    recommender_instance.fit(topK = optuna_trial.suggest_int("topK", 5, 1000),
                             shrink = optuna_trial.suggest_int("shrink", 0, 1000),
                             similarity =  optuna_trial.suggest_categorical("similarity", ["cosine", "jaccard", "adjusted", "asymmetric","pearson",  "dice" , "tversky",  "tanimoto"]),
                             normalize = optuna_trial.suggest_categorical("normalize", [True, False])
                            )

    result_df, _ = evaluator_validation.evaluateRecommender(recommender_instance)

    return result_df.loc[10]["MAP"]

In [62]:
class SaveResults(object):

    def __init__(self):
        self.results_df = pd.DataFrame()

    def __call__(self, optuna_study, optuna_trial):
        hyperparam_dict = optuna_trial.params.copy()
        hyperparam_dict["result"] = optuna_trial.values[0]

        self.results_df = self.results_df.append(hyperparam_dict, ignore_index=True)

In [71]:
optuna_study = optuna.create_study(direction="maximize")

save_results = SaveResults()

optuna_study.optimize(objective_function,
                      callbacks=[save_results],
                      n_trials = 50)

[I 2023-11-24 15:15:31,893] A new study created in memory with name: no-name-02eef95d-7ab6-4ffa-8887-01dc35f65f2f


ItemKNNCFRecommender: URM Detected 454 ( 3.6%) users with no interactions.
ItemKNNCFRecommender: URM Detected 348 ( 1.6%) items with no interactions.
Similarity column 22222 (100.0%), 2876.32 column/sec. Elapsed time 7.73 sec
EvaluatorHoldout: Processed 9941 (100.0%) in 12.96 sec. Users per second: 767


[I 2023-11-24 15:15:53,131] Trial 0 finished with value: 0.017172029258338415 and parameters: {'topK': 569, 'shrink': 528, 'similarity': 'asymmetric', 'normalize': True}. Best is trial 0 with value: 0.017172029258338415.


ItemKNNCFRecommender: URM Detected 454 ( 3.6%) users with no interactions.
ItemKNNCFRecommender: URM Detected 348 ( 1.6%) items with no interactions.


<ipython-input-62-280dc6e6b0e1>:10: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  self.results_df = self.results_df.append(hyperparam_dict, ignore_index=True)


...

[I 2023-11-24 15:29:01,251] Trial 48 finished with value: 0.02263621637502538 and parameters: {'topK': 281, 'shrink': 450, 'similarity': 'tversky', 'normalize': True}. Best is trial 28 with value: 0.028057196187666136.


ItemKNNCFRecommender: URM Detected 454 ( 3.6%) users with no interactions.
ItemKNNCFRecommender: URM Detected 348 ( 1.6%) items with no interactions.


<ipython-input-62-280dc6e6b0e1>:10: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  self.results_df = self.results_df.append(hyperparam_dict, ignore_index=True)


Similarity column 22222 (100.0%), 3948.49 column/sec. Elapsed time 5.63 sec
EvaluatorHoldout: Processed 9941 (100.0%) in 12.14 sec. Users per second: 819


[I 2023-11-24 15:29:19,319] Trial 49 finished with value: 0.021037490240035272 and parameters: {'topK': 220, 'shrink': 297, 'similarity': 'cosine', 'normalize': True}. Best is trial 28 with value: 0.028057196187666136.
<ipython-input-62-280dc6e6b0e1>:10: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  self.results_df = self.results_df.append(hyperparam_dict, ignore_index=True)


In [72]:
optuna_study.best_trial.params

{'topK': 73, 'shrink': 36, 'similarity': 'tversky', 'normalize': True}

In [73]:
recommender_instance = ItemKNNCFRecommender(urm_train + urm_validation)
recommender_instance.fit(**optuna_study.best_trial.params)

result_df, _ = evaluator_test.evaluateRecommender(recommender_instance)
result_df

ItemKNNCFRecommender: URM Detected 230 ( 1.8%) users with no interactions.
ItemKNNCFRecommender: URM Detected 93 ( 0.4%) items with no interactions.
Similarity column 22222 (100.0%), 3723.03 column/sec. Elapsed time 5.97 sec
EvaluatorHoldout: Processed 10486 (100.0%) in 10.01 sec. Users per second: 1048


,PRECISION,PRECISION_RECALL_MIN_DEN,RECALL,MAP,MAP_MIN_DEN,MRR,NDCG,F1,HIT_RATE,ARHR_ALL_HITS,NOVELTY,AVERAGE_POPULARITY,DIVERSITY_MEAN_INTER_LIST,DIVERSITY_HERFINDAHL,COVERAGE_ITEM,COVERAGE_ITEM_HIT,ITEMS_IN_GT,COVERAGE_USER,COVERAGE_USER_HIT,USERS_IN_GT,DIVERSITY_GINI,SHANNON_ENTROPY,RATIO_DIVERSITY_HERFINDAHL,RATIO_DIVERSITY_GINI,RATIO_SHANNON_ENTROPY,RATIO_AVERAGE_POPULARITY,RATIO_NOVELTY
cutoff,,,,,,,,,,,,,,,,,,,,,,,,,,,
10,0.082348,0.146702,0.123929,0.041817,0.07281,0.242712,0.132591,0.098947,0.475682,0.315258,0.004948,0.265236,0.960099,0.996001,0.307668,0.076636,0.801548,0.82972,0.394683,0.82972,0.047566,9.711469,0.996361,0.13656,0.748157,2.082066,0.279549


#Evaluate the Recommendation System

In [ ]:
def recall(recommendations: np.array, relevant_items: np.array) -> float:
    is_relevant = np.in1d(recommendations, relevant_items, assume_unique=True)

    recall_score = np.sum(is_relevant) / relevant_items.shape[0]

    return recall_score


def precision(recommendations: np.array, relevant_items: np.array) -> float:
    is_relevant = np.in1d(recommendations, relevant_items, assume_unique=True)

    precision_score = np.sum(is_relevant) / recommendations.shape[0]

    return precision_score

def mean_average_precision(recommendations: np.array, relevant_items: np.array) -> float:
    is_relevant = np.in1d(recommendations, relevant_items, assume_unique=True)

    precision_at_k = is_relevant * np.cumsum(is_relevant, dtype=np.float32) / (1 + np.arange(is_relevant.shape[0]))

    map_score = np.sum(precision_at_k) / np.min([relevant_items.shape[0], is_relevant.shape[0]])

    return map_score

def AP(recommended_items, relevant_items):

    is_relevant = np.in1d(recommended_items, relevant_items, assume_unique=True)

    # Cumulative sum: precision at 1, at 2, at 3 ...
    p_at_k = is_relevant * np.cumsum(is_relevant, dtype=np.float32) / (1 + np.arange(is_relevant.shape[0]))

    ap_score = np.sum(p_at_k) / np.min([relevant_items.shape[0], is_relevant.shape[0]])

    return ap_score

In [74]:
def evaluator(recommender: object, urm_train: sp.csr_matrix, urm_test: sp.csr_matrix):
    recommendation_length = 10
    accum_precision = 0
    accum_recall = 0
    accum_map = 0

    num_users = urm_train.shape[0]

    num_users_evaluated = 0
    num_users_skipped = 0
    for user_id in range(num_users):
        user_profile_start = urm_test.indptr[user_id]
        user_profile_end = urm_test.indptr[user_id+1]

        relevant_items = urm_test.indices[user_profile_start:user_profile_end]

        if relevant_items.size == 0:
            num_users_skipped += 1
            continue

        recommendations = recommender.recommend(user_id=user_id,
                                               at=recommendation_length,
                                               urm_train=urm_train,
                                               remove_seen=True)

        accum_precision += precision(recommendations, relevant_items)
        accum_recall += recall(recommendations, relevant_items)
        accum_map += mean_average_precision(recommendations, relevant_items)

        num_users_evaluated += 1


    accum_precision /= max(num_users_evaluated, 1)
    accum_recall /= max(num_users_evaluated, 1)
    accum_map /=  max(num_users_evaluated, 1)

    return accum_precision, accum_recall, accum_map, num_users_evaluated, num_users_skipped

In [75]:
accum_precision, accum_recall, accum_map, num_user_evaluated, num_users_skipped = evaluator(itemknn_recommender,
                                                                                            urm_train,
                                                                                            urm_test)

NameError: ignored

In [ ]:
accum_precision, accum_recall, accum_map, num_user_evaluated, num_users_skipped

#Prepare Submission

In [76]:
#forse ho sbagliato ad agggiungere anche il test set qui
best_shrink = 5
urm_train_validation = urm_train + urm_validation + urm_test
urm_train_validation

<12638x22222 sparse matrix of type '<class 'numpy.float64'>'
	with 478730 stored elements in Compressed Sparse Row format>

In [77]:
best_recommender = ItemKNNCFRecommender(urm_train_validation)
best_recommender.fit(**optuna_study.best_trial.params)

Similarity column 22222 (100.0%), 3139.52 column/sec. Elapsed time 7.08 sec


##Top popular for those users without info

In [78]:
toppop_recommender = TopPopRecommender()
toppop_recommender.fit(urm_train_validation)

In [79]:
users_to_recommend = np.array(data_target_users_test["user_id"])
len(users_to_recommend)

10882

In [80]:
mapping_to_item_id = dict(zip(interactions.mapped_item_id, interactions.item_id))

In [97]:
def prepare_submission(interactions: pd.DataFrame, users_to_recommend: np.array, urm_train: sp.csr_matrix, recommender: object, toppop_recommender: object):
    users_ids_and_mappings = interactions[interactions.user_id.isin(users_to_recommend)][['user_id', 'mapped_user_id']].drop_duplicates()
    items_ids_and_mappings = interactions[["item_id", "mapped_item_id"]].drop_duplicates()

    mapping_to_item_id = dict(zip(interactions.mapped_item_id, interactions.item_id))


    recommendation_length = 10
    submission = []
    for idx, row in users_ids_and_mappings.iterrows():
        user_id = row.user_id
        mapped_user_id = row.mapped_user_id

        recommendations = recommender.recommend(user_id_array=mapped_user_id, cutoff=10)

        submission.append((user_id, [mapping_to_item_id[item_id] for item_id in recommendations]))

    #assign top popular to the user without interactions
    users_without_info = np.setdiff1d(users_to_recommend,np.array(interactions[interactions.user_id.isin(users_to_recommend)][['user_id', 'mapped_user_id']].drop_duplicates().user_id))
    for user_id in users_without_info:
      recommendations = toppop_recommender.recommend(user_id, at = 10)
      submission.append((user_id, [mapping_to_item_id[item_id] for item_id in recommendations]))

    return submission

In [98]:
submission = prepare_submission(interactions, users_to_recommend, urm_train_validation, best_recommender, toppop_recommender)

In [99]:
submission

[(1, [101, 506, 403, 515, 36, 1546, 637, 977, 922, 869]),
 (2, [11, 47, 28, 50, 3, 2, 6, 4, 12, 5]),
 (26, [1089, 139, 254, 314, 1256, 1005, 474, 1620, 269, 41]),
 (36, [4, 3, 6, 15, 16, 131, 14, 20, 107, 48]),
 (41, [31, 55, 58, 486, 20, 4, 592, 1, 33, 9]),
 (47, [96, 214, 344, 211, 38, 154, 1635, 85, 15, 72]),
 (54, [19, 2, 6, 4, 1, 8, 20, 238, 24, 95]),
 (73, [21, 125, 85, 51, 154, 34, 109, 3, 32, 81]),
 (88, [99, 346, 1089, 139, 4, 3, 318, 563, 60, 2204]),
 (89, [138, 324, 344, 154, 10, 59, 433, 12673, 12672, 15014]),
 (95, [44, 4, 20, 26, 16, 6, 75, 88, 84, 15]),
 (100, [313, 764, 636, 99, 119, 11, 252, 3, 63, 108]),
 (101, [837, 2314, 754, 624, 1290, 828, 282, 1051, 1207, 1477]),
 (102, [2904, 2843, 2283, 2284, 3208, 2585, 2958, 7055, 1795, 5724]),
 (104, [3, 9, 4, 119, 2, 299, 1, 252, 8, 41]),
 (114, [17, 27, 50, 37, 5, 111, 42, 433, 102, 293]),
 (124, [346, 1342, 1198, 715, 34, 94, 2590, 1387, 2963, 1704]),
 (127, [41, 40, 618, 39, 55, 11, 42, 2, 47, 50]),
 (138, [11, 28, 50, 1

In [100]:
def write_submission(submissions):
    with open("./submission.csv", "w") as f:
        f.write("user_id,item_list\n")
        for user_id, items in submissions:
            f.write(f"{user_id},{' '.join([str(item) for item in items])}\n")


In [101]:
write_submission(submission)